In [1]:
# !pip install folium
# ! pip install ipywidgets
# !pip install geopy
# !pip install ACO-Pants

In [2]:
# !pip install competitions-scheduler

In [3]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [4]:
import random

# Generator function for list of matchups from a team_list
def games_from_list(team_list):
    for i in range(4):
        yield team_list[i], team_list[i+4]

# Function to apply rotation to list of teams as described in article
def rotate_list(team_list):
    team_list = [team_list[4]] + team_list[0:3] + team_list[5:8] + [team_list[3]]
    team_list[0], team_list[1] = team_list[1], team_list[0]
    return team_list

# Function to check if a list of games is valid
def checkValid(game_list):
    if len(set(game_list)) != len(game_list):
        return False
    for week in range(14):
        teams = set()
        this_week_games = game_list[week*4:week*4 + 4]
        for game in this_week_games:
            teams.add(game[0])
            teams.add(game[1])
        if len(teams) < 8:
            return False
    else:
        return True


# Generate list of teams & empty list of games played
teams = list(range(8))
games_played = []
week_add = []
game_week = []

# Optionally shuffle teams before generating schedule
random.shuffle(teams)

# For each week -
for week in range(14):
    # print(f"Week {week + 1}")

    # Get all the pairs of games from the list of teams.
    for pair in games_from_list(teams):
        # If the matchup has already been played:
        if pair in games_played:
            # Play the opposite match
            pair = pair[::-1]

        # Print the matchup and append to list of games.
        # print(f"{pair[0]} vs {pair[1]}")
        games_played.append(pair)
        week_add.append(pair)

    game_week.append(week_add)
    week_add = []

    # Rotate the list of teams
    teams = rotate_list(teams)

# Checks that the list of games is valid 
checkValid(games_played)

True

# 1

In [5]:
import folium 
import pandas as pd
import numpy as np
from geopy import distance
import pants

In [6]:
pesquisa = pd.read_excel('Municipios.xlsx')

In [7]:
times = pesquisa.query('NM_UF == "TOCANTINS"')
times.reset_index(drop=True, inplace=True)
times = times[['LONG', 'LAT', 'NM_LOCALIDADE']]
times

,LONG,LAT,NM_LOCALIDADE
0,-49.162438,-9.621791,ABREULÂNDIA
1,-47.470457,-6.547171,AGUIARNÓPOLIS
2,-48.941240,-11.303743,ALIANÇA DO TOCANTINS
3,-47.174172,-11.574523,ALMAS
4,-49.126811,-12.478815,ALVORADA
...,...,...,...
134,-47.427413,-6.325634,TOCANTINÓPOLIS
135,-48.206685,-8.980570,TUPIRAMA
136,-48.122996,-8.395530,TUPIRATINS
137,-47.956268,-6.850126,WANDERLÂNDIA


In [8]:
# times = pd.read_excel('brasileirão.xlsx')
listas = []
for i in range(len(times)):
    listas.append([times['LAT'][i], times['LONG'][i]])
times['info'] = listas

In [9]:
# Mapa do Brasil
mapa = folium.Map(location=[-15.77972, -46.92972],
                  zoom_start=4)

for i in range(len(times)):
    folium.CircleMarker(radius=5,
                        location=times['info'][i],
                        popup=times['NM_LOCALIDADE'][i],
                        color='#3186cc',
                        fill=True,
                        fill_color='#3186cc').add_to(mapa)
mapa.save('mapa.png')
mapa

In [10]:
# Corinthians até Recife
distance.distance((times['info'][7]), (times['info'][19])).km

305.5340790554539

In [11]:
# Corinthians até São Paulo
distance.distance((times['info'][7]), (times['info'][18])).km

479.5708710008139

In [12]:
def graphTSP(dados, numCities):
    cities = np.zeros((numCities, numCities), dtype=int)
    for i in range(numCities):
        for j in range(numCities):
            if (j > i):
                cities[i, j] = distance.distance((times['info'][i]),(times['info'][j])).km
            elif (j < i):
                cities[i, j] = cities[j, i]
    return cities.tolist()

In [13]:
numCities = len(times)
cities = graphTSP(times, numCities)
cities = cities[:5]
print('Número de cidades:', len(cities))
# print(cities)

Número de cidades: 5


In [14]:
def create_data_model(cities):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = cities
    data['num_vehicles'] = len(cities)//2
    data['depot'] = 0
    
    return data 

In [15]:
data = create_data_model(cities)
print(data['num_vehicles'])
print(data)

2
{'distance_matrix': [[0, 387, 187, 306, 316, 379, 385, 135, 280, 100, 372, 288, 340, 453, 218, 439, 480, 455, 467, 309, 215, 268, 52, 196, 131, 155, 167, 487, 411, 317, 250, 298, 491, 94, 213, 269, 57, 188, 108, 454, 352, 140, 107, 166, 357, 338, 19, 41, 190, 475, 128, 278, 312, 244, 94, 97, 293, 112, 233, 195, 205, 464, 179, 127, 340, 166, 138, 190, 89, 455, 273, 405, 57, 318, 445, 85, 63, 170, 46, 281, 278, 395, 234, 108, 168, 460, 368, 123, 107, 237, 376, 386, 67, 359, 231, 130, 274, 116, 241, 329, 90, 401, 216, 319, 145, 490, 140, 93, 233, 369, 318, 75, 141, 493, 333, 278, 181, 140, 232, 165, 388, 418, 281, 482, 360, 493, 279, 200, 475, 262, 427, 370, 350, 86, 411, 126, 177, 334, 361], [387, 0, 550, 556, 680, 70, 46, 381, 134, 339, 751, 108, 129, 122, 215, 708, 127, 691, 107, 79, 181, 128, 388, 215, 278, 232, 507, 159, 68, 173, 619, 115, 148, 406, 267, 562, 440, 200, 279, 700, 627, 274, 485, 528, 36, 566, 406, 347, 565, 176, 490, 645, 87, 623, 300, 294, 129, 277, 599, 530, 206, 8

In [28]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(cities)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        1000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # Time Limit
    search_parameters.time_limit.seconds = 120

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        x = get_solution(manager, routing, solution)
        
    else:
        print('No solution found !')


if __name__ == '__main__':
    main()

Objective: 87432
Route for vehicle 0:
 0 ->  1 -> 0
Distance of the route: 774m

Route for vehicle 1:
 0 ->  2 ->  4 ->  3 -> 0
Distance of the route: 858m

Maximum of the route distances: 858m


NameError: name 'get_solution' is not defined

In [17]:
re = main()

Objective: 87432
Route for vehicle 0:
 0 ->  1 -> 0
Distance of the route: 774m

Route for vehicle 1:
 0 ->  2 ->  4 ->  3 -> 0
Distance of the route: 858m

Maximum of the route distances: 858m


In [18]:
def DefaultRoutingSearchParameters() -> "operations_research::RoutingSearchParameters":
    return _pywrapcp.DefaultRoutingSearchParameters()

In [21]:
re

(<ortools.constraint_solver.pywrapcp.RoutingIndexManager; proxy of <Swig Object of type 'operations_research::RoutingIndexManager *' at 0x000001145EE13660> >,
 <ortools.constraint_solver.pywrapcp.RoutingModel; proxy of <Swig Object of type 'operations_research::RoutingModel *' at 0x000001145EE13390> >,
 <ortools.constraint_solver.pywrapcp.Assignment; proxy of <Swig Object of type 'operations_research::Assignment *' at 0x000001145EE19A80> >)